In [44]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Float
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from langchain_core.runnables.config import RunnableConfig
from azure.identity import DefaultAzureCredential
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from chromadb import PersistentClient
from chromadb.utils.embedding_functions import EmbeddingFunction
from transformers import AutoTokenizer, AutoModel


load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL", "sqlite:///example.db")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

CHROMA_DB_DIR = "./chroma_db"
COLLECTION_NAME = "schema_embeddings"
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"

Base = declarative_base()


# Definition of the Users table
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    age = Column(Integer)
    email = Column(String, unique=True, index=True)

    orders = relationship("Order", back_populates="user")

# Definition of the Food table
class Food(Base):
    __tablename__ = "food"

    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, unique=True, index=True)
    price = Column(Float)

    orders = relationship("Order", back_populates="food")

# Definition of the Orders table
class Order(Base):
    __tablename__ = "orders"

    id = Column(Integer, primary_key=True, index=True)
    food_id = Column(Integer, ForeignKey("food.id"))
    user_id = Column(Integer, ForeignKey("users.id"))

    user = relationship("User", back_populates="orders")
    food = relationship("Food", back_populates="orders")

## Functions for Agent

In [75]:
import os
from typing import List, Optional
from chromadb import PersistentClient
from dotenv import load_dotenv
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_openai import AzureChatOpenAI, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sqlalchemy import text, inspect
from langgraph.graph import StateGraph, END
import json
from langchain_core.prompts.chat import SystemMessage, HumanMessage
# Load environment variables
load_dotenv()

# Define the path to the ChromaDB directory
CHROMA_DB_DIR = os.getenv("CHROMA_DB_DIR", "chromadb")

class AgentState(TypedDict):
    question: str
    sql_query: Optional[str]
    query_result: Optional[str]
    query_rows: List[dict]
    current_user: str
    attempts: int
    relevance: str
    sql_error: bool
    context: List[str]
    chart_type: Optional[str]
    chart_title: Optional[str]
    

MAX_RETRIES = 3

def get_database_schema(engine):
    inspector = inspect(engine)
    schema = ""
    table_names = inspector.get_table_names()

    for table_name in table_names:
        schema += f"Table: {table_name}\n"
        for column in inspector.get_columns(table_name):
            col_name = column["name"]
            col_type = str(column["type"])
            if column.get("primary_key"):
                col_type += ", Primary Key"
            if column.get("foreign_keys"):
                fk = list(column["foreign_keys"])[0]
                col_type += f", Foreign Key to {fk.column.table.name}.{fk.column.name}"
            schema += f"- {col_name}: {col_type}\n"
        schema += "\n"
    print("Retrieved database schema.")
    return schema

class GetCurrentUser(BaseModel):
    current_user: str = Field(
        description="The name of the current user based on the provided user ID."
    )

def get_current_user(state: AgentState, config: RunnableConfig):
    print("Retrieving the current user based on user ID.")
    user_id = config["configurable"].get("current_user_id", None)
    if not user_id:
        state["current_user"] = "User not found"
        print("No user ID provided in the configuration.")
        return state

    session = SessionLocal()
    try:
        user = session.query(User).filter(User.id == int(user_id)).first()
        if user:
            state["current_user"] = user.name
            print(f"Current user set to: {state['current_user']}")
        else:
            state["current_user"] = "User not found"
            print("User not found in the database.")
    except Exception as e:
        state["current_user"] = "Error retrieving user"
        print(f"Error retrieving user: {str(e)}")
    finally:
        session.close()
    return state

class CheckRelevance(BaseModel):
    relevance: str = Field(
        description="Indicates whether the question is related to the database schema. 'relevant' or 'not_relevant'."
    )

from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai import ChatOpenAI



class ConvertToSQL(BaseModel):
    sql_query: str = Field(
        description="The SQL query corresponding to the user's natural language question."
    )

# Convert Natural Language to SQL

def convert_nl_to_sql(state, engine):
    question = state["question"]
    system_prompt = f"""
    Convert the question into an SQL query using this schema:
    
    Question: {question}
    Database Schema: {get_database_schema(engine)}
    user_id: {state.get("current_user_id", "None")}
    """
    
    llm = AzureChatOpenAI(
        temperature=0,
        azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
        openai_api_version=os.getenv("OPENAI_API_VERSION"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")
    )
    
    response = llm([SystemMessage(content=system_prompt)])
    response_content = response.content.strip()
    
    # Debugging: Print the raw response content
    print("Raw LLM Response Content:", response_content)
    
    # Remove backticks from the response content if present
    if response_content.startswith("```sql") and response_content.endswith("```"):
        response_content = response_content[6:-3].strip()
    
    # Debugging: Print the cleaned response content
    print("Cleaned LLM Response Content:", response_content)
    
    # Assign the SQL query to the state
    state["sql_query"] = response_content
    return state

def execute_sql(state: AgentState) -> AgentState:
    """Execute SQL query with safety checks"""
    print("[DEBUG] Executing SQL query")
    session = SessionLocal()
    state["sql_error"] = False
    
    try:
        if not state["sql_query"] or not state["sql_query"].lower().startswith("select"):
            raise ValueError("Only SELECT queries are allowed")
            
        result = session.execute(text(state["sql_query"]))
        
        if "select" in state["sql_query"].lower():
            columns = result.keys()
            rows = [dict(zip(columns, row)) for row in result.fetchall()]
            state.update({
                "query_rows": rows,
                "query_result": f"Retrieved {len(rows)} records"
            })
        else:
            session.commit()
            state["query_result"] = "Operation completed successfully"
            
    except Exception as e:
        state.update({
            "sql_error": True,
            "query_result": f"SQL Error: {str(e)}"
        })
    finally:
        session.close()
    
    print("[DEBUG] SQL execution result:", state["query_result"])
    return state


# Regenerate SQL Query
def regenerate_query(state: AgentState):
    if state["attempts"] >= MAX_RETRIES:
        state["query_result"] = "Max retries reached. Unable to process your request."
        state["sql_query"] = None  # Prevent further execution
        return state

    # Regenerate question
    question = state["question"]
    system_message = "Reformulate the question to make it SQL-compatible."
    human_message = f"Original Question: {question}"

    llm = AzureChatOpenAI(
        temperature=0,
        azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
        openai_api_version=os.getenv("OPENAI_API_VERSION"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME"),
    )
    response = llm([SystemMessage(content=system_message), HumanMessage(content=human_message)])
    state["question"] = response.content.strip()
    state["attempts"] += 1
    return state



def generate_human_readable_answer(state: AgentState) -> AgentState:
    """Generate natural language response from query results"""
    print("[DEBUG] Generating human-readable answer")
    
    components = {
        "user": state.get("current_user", "User"),
        "error": state.get("sql_error", False),
        "result": state.get("query_result", ""),
        "rows": state.get("query_rows", []),
        "chart_type": state.get("chart_type"),
        "chart_title": state.get("chart_title")
    }

    # Base template
    template = f"Hello {components['user']}, "
    
    # Error handling
    if components["error"]:
        template += f"we encountered an error: {components['result']}"
    # Visualization case
    elif components["chart_type"]:
        template += (
            f"I've created a {components['chart_type']} chart "
            f"titled '{components['chart_title']}'. "
            f"{components['result']}"
        )
    # Data results case
    elif components["rows"]:
        sample = components["rows"][:3]
        template += f"found {len(components['rows'])} records. Sample data: {sample}"
    # No results case
    else:
        template += "no matching records were found."
    
    state["query_result"] = template
    print("[DEBUG] Generated answer:", template)
    return state




class RewrittenQuestion(BaseModel):
    question: str = Field(description="The rewritten question.")



def generate_funny_response(state: AgentState):
    print("Generating a funny response for an unrelated question.")
    system = """You are a charming and funny assistant who responds in a playful manner.
    """
    human_message = "I can not help with that, but doesn't asking questions make you hungry? You can always order something delicious."
    funny_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", human_message),
        ]
    )
    llm = AzureChatOpenAI(
        temperature=0.8,
        azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
        openai_api_version=os.getenv("OPENAI_API_VERSION"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")
    )

    funny_response = funny_prompt | llm | StrOutputParser()
    message = funny_response.invoke({})
    state["query_result"] = message
    print("Generated funny response.")
    return state

def end_max_iterations(state: AgentState):
    state["query_result"] = "Please try again."
    print("Maximum attempts reached. Ending the workflow.")
    return state

def relevance_router(state: AgentState):
    relevance = state.get("relevance", "").lower()
    if relevance == "relevant":
        return "retrieve_context" 
    else:
        return "generate_funny_response"


def check_attempts_router(state: AgentState):
    attempts = state.get("attempts", 0)
    if attempts < 10:
        return "convert_nl_to_sql"
    else:
        return "end_max_iterations"


def execute_sql_router(state: AgentState):
    if state.get("sql_query") is None or state.get("sql_error", False):
        if state["attempts"] >= MAX_RETRIES:
            return "end_max_iterations"
        return "regenerate_query"
    elif state.get("query_rows"):
        return "generate_visualization"
    else:
        return "generate_human_readable_answer"


    
def merge_question_updates(state):
    """
    Consolidates updates to the 'question' key.
    """
    state["merged_question"] = state.get("question", "")
    return state







In [77]:
from typing import Dict, List
import torch



class ThaiBGEEmbeddingFunction(EmbeddingFunction):
    """
    Embedding function with proper pooling and Thai language support
    """
    def __init__(self, model_name= "BAAI/bge-m3"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.eval()  # Disable dropout for evaluation

    def __call__(self, texts: List[str]) -> List[List[float]]:
        """
        Generate embeddings for input texts
        Args:
            texts (List[str]): List of strings to embed
        Returns:
            List[List[float]]: List of embeddings
        """
        if isinstance(texts, str):  # Ensure input is a list
            texts = [texts]
        
        inputs = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=512
        )
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0]  # CLS pooling
        return embeddings.cpu().numpy().tolist()

def process_schema_to_graph_and_embeddings(schema: Dict, chroma_collection, embedding_model) -> nx.DiGraph:
    """
    Process database schema into a graph structure and store embeddings in ChromaDB,
    using M-Schema format for enhanced metadata.
    """
    graph = nx.DiGraph()
    entries = []

    for table_name, details in schema.items():
        # Add table as a node with enriched metadata
        graph.add_node(
            table_name,
            type="table",
            description=details.get("description", ""),
            primary_keys=details.get("primary_keys", []),
            examples=details.get("examples", {})
        )

        # Process columns with M-Schema metadata
        for column in details["columns"]:
            column_id = f"{table_name}.{column['name']}"
            
            # Safely get examples for the column and serialize them
            examples = column.get("examples", [])
            examples_serialized = ", ".join(map(str, examples)) if examples else "None"
            
            graph.add_node(
                column_id,
                type="column",
                table=table_name,
                data_type=column["type"],
                primary_key=column.get("primary_key", False),
                examples=examples_serialized  # Store serialized examples
            )
            graph.add_edge(table_name, column_id, relation="has_column")

            # Add column embedding entry
            column_text = (
                f"Column: {column['name']} ({column['type']}), "
                f"Primary Key: {column.get('primary_key', False)}, "
                f"Examples: {examples_serialized}"
            )
            embedding = embedding_model([column_text])[0]

            entries.append({
                "id": column_id,
                "text": column_text,
                "embedding": embedding,
                "metadata": {
                    "type": "column",
                    "table": table_name,
                    "data_type": column["type"],
                    "primary_key": column.get("primary_key", False),
                    "examples": examples_serialized  # Store serialized examples in metadata
                }
            })

        # Process relationships with enriched metadata
        for relation in details.get("relationships", []):
            rel_id = f"{table_name}.{relation['source_column']}->{relation['target_table']}.{relation['target_column']}"
            
            graph.add_edge(
                table_name,
                relation["target_table"],
                relation="foreign_key",
                source_column=relation["source_column"],
                target_column=relation["target_column"]
            )
            
            # Relationship text for embeddings
            rel_text = (
                f"Foreign Key: {table_name}.{relation['source_column']} ‚Üí "
                f"{relation['target_table']}.{relation['target_column']}, "
                f"Maps to: {relation.get('maps_to', '')}"
            )
            embedding = embedding_model([rel_text])[0]

            entries.append({
                "id": rel_id,
                "text": rel_text,
                "embedding": embedding,
                "metadata": {
                    "type": "relationship",
                    "source_table": table_name,
                    "source_column": relation["source_column"],
                    "target_table": relation["target_table"],
                    "target_column": relation["target_column"],
                    "maps_to": relation.get("maps_to", "")
                }
            })

    # Debug: Print sample metadata for verification
    if entries:
        print(f"Sample Metadata Entry: {entries[0]['metadata']}")

    # Store embeddings in ChromaDB
    if entries:
        chroma_collection.add(
            ids=[entry["id"] for entry in entries],
            documents=[entry["text"] for entry in entries],
            embeddings=[entry["embedding"] for entry in entries],
            metadatas=[entry["metadata"] for entry in entries]
        )
    
    return graph


# -------------------------- ENHANCED RETRIEVAL ------------------------- #
def retrieve_context(state: AgentState):
    state["context"] = hybrid_search(
        state["question"],
        schema,
        chroma_collection,
        embedding_model,
        graph,
        max_results=15
    )
    return state


from sqlalchemy.sql import text  # Import text for raw SQL queries

def extract_database_schema(engine):
    inspector = inspect(engine)
    schema = {}

    # Create a connection for executing SQL queries
    with engine.connect() as connection:
        tables = inspector.get_table_names()

        for table_name in tables:
            schema[table_name] = {"columns": [], "relationships": [], "examples": {}}
            
            # Extract columns
            columns = inspector.get_columns(table_name)
            for column in columns:
                # Add column details with examples
                examples = []  # Retrieve examples from the database
                try:
                    # Use sqlalchemy.text to create a valid SQL query object
                    query = text(f"SELECT {column['name']} FROM {table_name} LIMIT 3")
                    examples = [row[0] for row in connection.execute(query).fetchall()]
                except Exception as e:
                    print(f"Failed to fetch examples for {table_name}.{column['name']}: {e}")
                
                schema[table_name]["columns"].append({
                    "name": column["name"],
                    "type": str(column["type"]).split("(")[0],
                    "examples": examples
                })

            # Extract foreign keys
            foreign_keys = inspector.get_foreign_keys(table_name)
            for fk in foreign_keys:
                schema[table_name]["relationships"].append({
                    "source_column": fk["constrained_columns"][0],
                    "target_table": fk["referred_table"],
                    "target_column": fk["referred_columns"][0]
                })

    return schema

def keyword_search(query: str, schema: Dict) -> List[str]:
    """
    Perform keyword search to find schema elements directly matching query keywords.
    Args:
        query (str): User's natural language query.
        schema (Dict): Extracted database schema.
    Returns:
        List[str]: List of schema elements matching the query keywords.
    """
    keywords = query.lower().split()  # Split the query into individual words
    matches = []

    for table, details in schema.items():
        # Match table names
        if any(keyword in table.lower() for keyword in keywords):
            matches.append(table)

        # Match column names
        for column in details["columns"]:
            if any(keyword in column["name"].lower() for keyword in keywords):
                matches.append(f"{table}.{column['name']}")

    return matches



def semantic_search(query: str, chroma_collection, embedding_model, max_results: int = 20) -> List[str]:
    """
    Perform semantic search to retrieve schema elements based on embeddings.
    Args:
        query (str): User's natural language query.
        chroma_collection: ChromaDB collection.
        embedding_model: Embedding model for generating embeddings.
        max_results (int): Maximum number of results to return.
    Returns:
        List[str]: List of schema elements matching the query semantically.
    """
    # Generate embedding for the query
    query_embedding = embedding_model([query])[0]
    
    # Query ChromaDB
    results = chroma_collection.query(query_embeddings=[query_embedding], n_results=max_results)
    
    # Extract IDs of matching schema elements
    return results["ids"][0]


def hybrid_search(
    query: str,
    schema: Dict,
    chroma_collection,
    embedding_model,
    graph: nx.DiGraph,
    max_results: int = 20
) -> List[str]:
    """
    Combine keyword search and semantic search for hybrid retrieval.
    Args:
        query (str): User's natural language query.
        schema (Dict): Extracted database schema.
        chroma_collection: ChromaDB collection.
        embedding_model: Embedding model for generating embeddings.
        max_results (int): Maximum number of results to return.
    Returns:
        List[str]: Combined results from keyword and semantic search.
    """
    # Perform keyword search
    keyword_matches = keyword_search(query, schema)

    # Perform semantic search
    semantic_matches = semantic_search(query, chroma_collection, embedding_model, max_results)

    # Combine and deduplicate results
    combined_matches = list(set(keyword_matches + semantic_matches))

    # Limit results to max_results
    return combined_matches[:max_results]


def assemble_prompt(query: str, context: List[str], graph: nx.DiGraph) -> str:
    """
    Build a structured and optimized prompt with schema context, highlighting relevant parts
    and including enriched metadata like examples and primary keys.
    """
    prompt_lines = [
        "### Task",
        "Translate the following natural language query into an SQL query:",
        "",
        f"### Query\n{query}",
        "",
        "### Relevant Schema Information"
    ]

    # Collect relevant tables and their metadata
    relevant_tables = set()
    for node in context:
        try:
            if graph.nodes[node]["type"] == "table":
                relevant_tables.add(node)
            elif graph.nodes[node]["type"] == "column":
                relevant_tables.add(graph.nodes[node]["table"])
        except KeyError:
            continue  # Skip nodes that lack necessary attributes

    # Add details for relevant tables and columns
    for table in relevant_tables:
        table_metadata = graph.nodes[table]
        primary_keys = table_metadata.get("primary_keys", [])
        examples = table_metadata.get("examples", {})
        prompt_lines.append(f"\nTable: {table}")
        if primary_keys:
            prompt_lines.append(f"  - Primary Keys: {', '.join(primary_keys)}")
        if examples:
            prompt_lines.append(f"  - Examples: {examples}")

        # Add column details
        for neighbor in graph.neighbors(table):
            try:
                if graph.nodes[neighbor]["type"] == "column":
                    column_metadata = graph.nodes[neighbor]
                    column_name = neighbor.split(".")[1]
                    column_type = column_metadata["data_type"]
                    primary_key = column_metadata.get("primary_key", False)
                    column_examples = column_metadata.get("examples", [])

                    # Highlight relevant columns
                    if neighbor in context:
                        prompt_lines.append(
                            f"  - {column_name} ({column_type}) [RELEVANT] - "
                            f"{'Primary Key' if primary_key else ''} "
                            f"{f'Examples: {column_examples}' if column_examples else ''}"
                        )
                    else:
                        prompt_lines.append(
                            f"  - {column_name} ({column_type}) - "
                            f"{'Primary Key' if primary_key else ''} "
                            f"{f'Examples: {column_examples}' if column_examples else ''}"
                        )
            except KeyError:
                continue

    # Add relevant relationships
    prompt_lines.append("\n### Relationships")
    for edge in graph.edges(data=True):
        if edge[2].get("relation") == "foreign_key":
            source = edge[0]
            target = edge[1]
            source_col = edge[2]["source_column"]
            target_col = edge[2]["target_column"]
            maps_to = edge[2].get("maps_to", "")
            relationship_examples = edge[2].get("examples", [])
            if source in relevant_tables or target in relevant_tables:
                prompt_lines.append(
                    f"  - {source} ({source_col}) ‚Üí {target} ({target_col}) [RELEVANT] "
                    f"{f'Maps to: {maps_to}' if maps_to else ''} "
                    f"{f'Examples: {relationship_examples}' if relationship_examples else ''}"
                )

    # Final instructions
    prompt_lines.append("\n### SQL Query")
    prompt_lines.append("-- Write the SQL query below this line --")
    print(f"Assemble Prompt: {prompt_lines}")
    return "\n".join(prompt_lines)


COLLECTION_NAME = 'schema_embeddings'

def setup_chroma_db():
    """
    Initialize ChromaDB and ensure collection matches embedding dimension
    """
    schema = extract_database_schema(engine)
    
    chroma_client = PersistentClient(path=CHROMA_DB_DIR)
    try:
        # Try to get the collection
        collection = chroma_client.get_collection(COLLECTION_NAME)
    except Exception:
        # Create collection with correct embedding function and dimension
        collection = chroma_client.create_collection(
            name=COLLECTION_NAME,
            embedding_function=ThaiBGEEmbeddingFunction()
        )
    return chroma_client, collection



schema = extract_database_schema(engine)
chroma_client, chroma_collection = setup_chroma_db()
embedding_model = ThaiBGEEmbeddingFunction()
graph = process_schema_to_graph_and_embeddings(schema, chroma_collection, embedding_model)

from langchain_core.prompts.chat import ChatPromptTemplate

def check_relevance(state: AgentState, config: dict):
    """Determine if the user's question is relevant to the database schema using context-based retrieval."""
    question = state["question"]
    
    # Retrieve relevant schema context using hybrid search
    state["context"] = hybrid_search(
        question,
        schema,  # Extracted database schema
        chroma_collection,
        embedding_model,
        graph,
        max_results=15  # Retrieve top 15 relevant elements
    )

    # If no relevant schema elements were found, mark as 'not_relevant'
    if not state["context"]:
        state["relevance"] = "not_relevant"
        print("[DEBUG] No relevant schema context found. Marking as not relevant.")
        return state

    # Construct prompt with retrieved schema context
    system_prompt = f"""
    You are an intelligent assistant determining if a given question is related to a database schema.

    - If the question is **clearly related** to the database (queries about data, tables, or relationships), respond `"relevant"`.
    - If the question is **unrelated** (about the weather, general facts, jokes, etc.), respond `"not_relevant"`.

    **Database Schema Context (Most Relevant Retrieved Entries):**
    {state["context"]}

    **Example Scenarios:**
    - User Question: "What is the weather today?"
      Response: "not_relevant"
    - User Question: "Show me all coupons and their discount."
      Response: "relevant"

    **User Question:**
    {question}
    """

    # LLM call to determine relevance
    llm = AzureChatOpenAI(
        temperature=0,
        azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
        openai_api_version=os.getenv("OPENAI_API_VERSION"),
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")
    )

    response = llm([SystemMessage(content=system_prompt)])

    # Extract response and update state
    state["relevance"] = response.content.strip().lower()
    print(f"[DEBUG] LLM determined relevance as: {state['relevance']}")

    return state


Insert of existing embedding ID: coupons.id
Insert of existing embedding ID: coupons.code
Insert of existing embedding ID: coupons.discount_percentage
Insert of existing embedding ID: coupons.expiration_date
Insert of existing embedding ID: discounts.id
Insert of existing embedding ID: discounts.food_id
Insert of existing embedding ID: discounts.discount_percentage
Insert of existing embedding ID: discounts.food_id->food.id
Insert of existing embedding ID: employees.id
Insert of existing embedding ID: employees.name
Insert of existing embedding ID: employees.position
Insert of existing embedding ID: employees.salary
Insert of existing embedding ID: food.id
Insert of existing embedding ID: food.name
Insert of existing embedding ID: food.price
Insert of existing embedding ID: food_category.id
Insert of existing embedding ID: food_category.name
Insert of existing embedding ID: food_category_mapping.id
Insert of existing embedding ID: food_category_mapping.food_id
Insert of existing embedd

Sample Metadata Entry: {'type': 'column', 'table': 'coupons', 'data_type': 'INTEGER', 'primary_key': False, 'examples': '3, 4, 9'}


## Debugging

In [15]:
state = {"question": "show me all coupons, and hilight minumum and maximum value", "sql_query": ""}
convert_nl_to_sql(state, engine)
print("Generated SQL:", state["sql_query"])


Retrieved database schema.
Raw LLM Response Content: ```sql
SELECT 
    code, 
    discount_percentage, 
    expiration_date,
    CASE 
        WHEN discount_percentage = (SELECT MIN(discount_percentage) FROM coupons) THEN 'Minimum Value'
        WHEN discount_percentage = (SELECT MAX(discount_percentage) FROM coupons) THEN 'Maximum Value'
        ELSE ''
    END AS value_status
FROM 
    coupons;
```
Cleaned LLM Response Content: SELECT 
    code, 
    discount_percentage, 
    expiration_date,
    CASE 
        WHEN discount_percentage = (SELECT MIN(discount_percentage) FROM coupons) THEN 'Minimum Value'
        WHEN discount_percentage = (SELECT MAX(discount_percentage) FROM coupons) THEN 'Maximum Value'
        ELSE ''
    END AS value_status
FROM 
    coupons;
Generated SQL: SELECT 
    code, 
    discount_percentage, 
    expiration_date,
    CASE 
        WHEN discount_percentage = (SELECT MIN(discount_percentage) FROM coupons) THEN 'Minimum Value'
        WHEN discount_percenta

In [5]:
state = {"sql_query": "SELECT * FROM coupons", "query_rows": [], "query_result": ""}
session = SessionLocal()
execute_sql(state, session)
print("Query Result:", state["query_result"])
print("Query Rows:", state["query_rows"])


[DEBUG] Raw Result: <sqlalchemy.engine.cursor.CursorResult object at 0x1059fd480>
[DEBUG] Rows: [(1, 'COUPONmBYy', 30.74, 1879459200000), (2, 'COUPONyLqw', 25.96, 1830816000000), (3, 'COUPONAmyC', 45.97, '2025-04-03'), (4, 'COUPONHrUX', 16.34, '2025-09-16'), (5, 'COUPONiozW', 9.92, '2025-08-25'), (6, 'COUPONvWwN', 16.96, '2025-06-27'), (7, 'COUPONPiTu', 9.9, '2025-02-18'), (8, 'COUPONUqyN', 21.92, '2025-05-03'), (9, 'COUPONOpbb', 8.87, '2025-08-15'), (10, 'COUPONVmFZ', 12.84, '2025-12-22')]
[DEBUG] Columns: RMKeyView(['id', 'code', 'discount_percentage', 'expiration_date'])
Query Result: Query executed successfully. Found 10 rows.
Query Rows: [{'id': 1, 'code': 'COUPONmBYy', 'discount_percentage': 30.74, 'expiration_date': 1879459200000}, {'id': 2, 'code': 'COUPONyLqw', 'discount_percentage': 25.96, 'expiration_date': 1830816000000}, {'id': 3, 'code': 'COUPONAmyC', 'discount_percentage': 45.97, 'expiration_date': '2025-04-03'}, {'id': 4, 'code': 'COUPONHrUX', 'discount_percentage': 16.

In [6]:
import os

state = {"question": "show me all orders", "sql_query": ""}
convert_nl_to_sql(state, engine)
print("Generated SQL:", state["sql_query"])

Retrieved database schema.
Raw LLM Response Content: ```sql
SELECT * FROM orders;
```
Cleaned LLM Response Content: SELECT * FROM orders;
Generated SQL: SELECT * FROM orders;


## Visualization

In [55]:
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from pydantic import BaseModel, Field, ValidationError
from langchain.schema import SystemMessage, HumanMessage
from langchain_openai import AzureChatOpenAI
# ------------------------ Visualization Components ------------------------ #

class VisualizationDetails(BaseModel):
    chart_type: str = Field(description="Type of chart (bar, pie, line)")
    x_column: str = Field(description="Column for x-axis/labels")
    y_column: str = Field(description="Column for y-axis/values")
    title: str = Field(description="Chart title")

def generate_visualization(state: AgentState) -> AgentState:
    """Generate data visualization from query results"""
    print("[DEBUG] Starting visualization generation")
    
    query_rows = state.get("query_rows", [])
    if not query_rows:
        state["query_result"] = "No data available for visualization."
        return state

    try:
        df = pd.DataFrame(query_rows)
        if len(df) < 1:
            raise ValueError("Empty DataFrame")

        # Get visualization parameters from LLM
        system_msg = f"""Analyze this data and recommend visualization:
        Columns: {list(df.columns)}
        First 3 rows: {df.head(3).to_dict(orient='records')}
        User Question: {state['question']}
        Respond in JSON format with: chart_type, x_column, y_column, title"""
        
        llm = AzureChatOpenAI(
            temperature=0,
            azure_endpoint=os.getenv("OPENAI_API_ENDPOINT"),
            api_key=os.getenv("OPENAI_API_KEY"),
            deployment_name=os.getenv("OPENAI_DEPLOYMENT_NAME")
        )
        
        response = llm.invoke([SystemMessage(content=system_msg)])
        print("[DEBUG] LLM Visualization Response:", response.content)

        # Parse and validate response
        response_content = response.content.strip("```json").strip("```").strip()
        vis_details = json.loads(response_content)
        
        # Key normalization
        key_mapping = {
            "x_axis_column": "x_column",
            "y_axis_column": "y_column",
            "labels_column": "x_column",
            "sizes_column": "y_column"
        }
        vis_details = {key_mapping.get(k, k): v for k, v in vis_details.items()}
        
        validated = VisualizationDetails.model_validate(vis_details)
        state.update({
            "chart_type": validated.chart_type,
            "chart_title": validated.title
        })

        # Validate DataFrame columns
        if validated.x_column not in df.columns:
            raise ValueError(f"Missing x-axis column: {validated.x_column}")
        if validated.y_column not in df.columns:
            raise ValueError(f"Missing y-axis column: {validated.y_column}")

        # Generate plot
        plt.figure(figsize=(10, 6))
        plot_methods = {
            "bar": lambda: df.groupby(validated.x_column)[validated.y_column].sum().plot(kind="bar"),
            "pie": lambda: df.groupby(validated.x_column)[validated.y_column].sum().plot(kind="pie", autopct="%1.1f%%"),
            "line": lambda: df.groupby(validated.x_column)[validated.y_column].sum().plot(kind="line"),
            "hist": lambda: df[validated.y_column].plot(kind="hist")
        }
        
        if validated.chart_type not in plot_methods:
            raise ValueError(f"Unsupported chart type: {validated.chart_type}")
        
        plot_methods[validated.chart_type]()
        plt.title(validated.title)
        plt.tight_layout()
        
        image_path = "query_visualization.png"
        plt.savefig(image_path)
        plt.close()
        
        state["query_result"] = f"Visualization saved at {image_path}"

    except json.JSONDecodeError as e:
        state["query_result"] = f"Visualization error: Invalid LLM response - {str(e)}"
    except ValidationError as e:
        state["query_result"] = f"Visualization error: Invalid parameters - {str(e)}"
    except Exception as e:
        state["query_result"] = f"Visualization error: {str(e)}"
    
    print("[DEBUG] Visualization result:", state["query_result"])
    return state


In [31]:
state = {
    "query_rows": [
        {"coupon_id": 1, "coupon_code": "SAVE10", "discount": 20, "expiry_date": "2024-12-31"},
        {"coupon_id": 2, "coupon_code": "FREESHIP", "discount": 35, "expiry_date": "2022-12-31"},
        {"coupon_id": 3, "coupon_code": "HOLIDAY15", "discount": 65, "expiry_date": "2027-12-31"},

        
    ],
    "question": "Show me all coupons, with their discount, sort them by discount amount",
}
generate_visualization(state)
print("Visualization Result:", state["query_result"])


[DEBUG] LLM Response Content: ```json
{
    "chart_type": "bar",
    "x_axis_column": "coupon_code",
    "y_axis_column": "discount",
    "title": "Coupons Sorted by Discount Amount"
}
```
Visualization saved at query_visualization.png.
Visualization Result: Visualization saved at query_visualization.png.


## Workflow

In [56]:
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("get_current_user", get_current_user)
workflow.add_node("check_relevance", check_relevance)
workflow.add_node("retrieve_context", retrieve_context)
workflow.add_node("convert_nl_to_sql", lambda state: convert_nl_to_sql(state, engine))
workflow.add_node("execute_sql", execute_sql)
workflow.add_node("generate_visualization", generate_visualization)
workflow.add_node("regenerate_query", regenerate_query)
workflow.add_node("generate_human_readable_answer", generate_human_readable_answer)
workflow.add_node("generate_funny_response", generate_funny_response)
workflow.add_node("end_max_iterations", end_max_iterations)

# Define edges
workflow.set_entry_point("get_current_user")
workflow.add_edge("get_current_user", "check_relevance")

# Relevance decision
workflow.add_conditional_edges(
    "check_relevance",
    relevance_router,
    {
        "retrieve_context": "retrieve_context",
        "generate_funny_response": "generate_funny_response"
    }
)

# Contextual SQL path
workflow.add_edge("retrieve_context", "convert_nl_to_sql")
workflow.add_edge("convert_nl_to_sql", "execute_sql")

# SQL execution outcomes
workflow.add_conditional_edges(
    "execute_sql",
    execute_sql_router,
    {
        "generate_human_readable_answer": "generate_human_readable_answer",
        "regenerate_query": "regenerate_query",
        "generate_visualization": "generate_visualization"
    }
)

# Regeneration logic
workflow.add_conditional_edges(
    "regenerate_query",
    check_attempts_router,
    {
        "convert_nl_to_sql": "convert_nl_to_sql",
        "end_max_iterations": "end_max_iterations"
    }
)

# Visualization path
workflow.add_edge("generate_visualization", "generate_human_readable_answer")

# Terminal nodes
workflow.add_edge("generate_human_readable_answer", END)
workflow.add_edge("generate_funny_response", END)
workflow.add_edge("end_max_iterations", END)

# Compile the workflow
app = workflow.compile()

In [49]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
except:
    pass

In [21]:
# save the graph
save_path = "workflow_graph.png"
graph = app.get_graph(xray=True).draw_mermaid_png()
with open(save_path, "wb") as f:
    f.write(graph)

In [50]:
fake_config = {"configurable": {"current_user_id": "5"},"recursion_limit": 50}

In [51]:
def get_orders_for_user(user_id: int):
    session = SessionLocal()
    try:
        orders = (
            session.query(Order)
            .join(Food, Order.food_id == Food.id)
            .filter(Order.user_id == user_id)
            .all()
        )
        if not orders:
            print(f"No orders found for user_id {user_id}.")
            return []

        order_list = []
        for order in orders:
            order_info = {
                "order_id": order.id,
                "food_name": order.food.name,
                "price": order.food.price
            }
            order_list.append(order_info)

        return order_list
    except Exception as e:
        print(f"Error retrieving orders for user_id {user_id}: {e}")
        return []
    finally:
        session.close()

orders = get_orders_for_user(5)
orders

[{'order_id': 8, 'food_name': 'Street_229709', 'price': 28.07},
 {'order_id': 54, 'food_name': 'Spaghetti Carbonara', 'price': 41.81},
 {'order_id': 55, 'food_name': 'Spaghetti Carbonara', 'price': 41.81}]

In [65]:
user_question_1 = "show me all coupons and their discount"
result_1 = app.invoke({"question": user_question_1, **fake_config})
print(result_1["query_result"])


Retrieving the current user based on user ID.
No user ID provided in the configuration.
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Ke

In [66]:
user_question_2 = "Tell me about Bangkok."
result_2 = app.invoke({"question": user_question_2, "attempts": 0}, config=fake_config)
print("Result:", result_2["query_result"])

Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [67]:
user_question_3 = "Find top 5 orders by price."
result_3 = app.invoke({"question": user_question_3, "attempts": 0}, config=fake_config)
print("Result:", result_3["query_result"])

Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [82]:
user_question_1 = '''Show me top 5 most cheap foods, use a pie chart.'''
result_1 = app.invoke({"question": user_question_1, "attempts": 0}, config=fake_config)
print("Result:", result_1["query_result"])


Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [83]:
user_question_1 = '''Show me top 5 users who has highest points.'''
result_1 = app.invoke({"question": user_question_1, "attempts": 0}, config=fake_config)
print("Result:", result_1["query_result"])


Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [86]:
user_question_1 = '''Show me Average Salary by position, use a histogram.'''
result_1 = app.invoke({"question": user_question_1, "attempts": 0}, config=fake_config)
print("Result:", result_1["query_result"])


Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [89]:
user_question_1 = '''Analyze ordered foods, use a line chart.'''
result_1 = app.invoke({"question": user_question_1, "attempts": 0}, config=fake_config)
print("Result:", result_1["query_result"])


Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo

In [103]:
user_question_1 = ''' Analyze food's discount percentage, and show it in a pie chart.'''
result_1 = app.invoke({"question": user_question_1, "attempts": 0}, config=fake_config)
print("Result:", result_1["query_result"])


Retrieving the current user based on user ID.
Current user set to: William Walton
Retrieved database schema.
Database schema: Table: coupons
- id: INTEGER, Primary Key
- code: VARCHAR
- discount_percentage: FLOAT
- expiration_date: DATE

Table: discounts
- id: INTEGER, Primary Key
- food_id: INTEGER
- discount_percentage: FLOAT

Table: employees
- id: INTEGER, Primary Key
- name: VARCHAR
- position: VARCHAR
- salary: FLOAT

Table: food
- id: INTEGER, Primary Key
- name: VARCHAR
- price: FLOAT

Table: food_category
- id: INTEGER, Primary Key
- name: VARCHAR

Table: food_category_mapping
- id: INTEGER, Primary Key
- food_id: INTEGER
- category_id: INTEGER

Table: ingredients
- id: INTEGER, Primary Key
- name: VARCHAR
- cost_per_unit: FLOAT

Table: inventory
- id: INTEGER, Primary Key
- ingredient_id: INTEGER
- stock_level: FLOAT

Table: loyalty_program
- id: INTEGER, Primary Key
- user_id: INTEGER
- points: INTEGER
- membership_tier: VARCHAR

Table: orders
- id: INTEGER, Primary Key
- fo